<a href="https://colab.research.google.com/github/osmarbraz/coebert/blob/main/AjusteFinoCSTNews_AvaliacaoMoodle_v1_C_SB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Ajuste fino do conjunto de dados CSTNews usando BERT e validação com os dados do OnlineEduc 1.0


Realiza o ajuste do MCL BERT pré-treinado usando o conjunto de dados CSTNews e a avaliação com o conjunto de dados OnlineEduc 1.0.

- Realiza o ajuste fino nos dados dos dados CSTNEWS.
- Utiliza Lotes Inteligentes para otimizar o tempo de execução de treinamento.
- Divide o dataset em 70% para treino e 30% para avaliação.
- Salva o modelo ajustado para reaproveitamento,
- A seção 2 - parametrização define os argumentos da execução.

----------------------------

**Link biblioteca Transformers:**
https://github.com/huggingface/transformers

**Artigo original BERT:**
https://arxiv.org/pdf/1506.06724.pdf

**Artigo padding dinâmico:**
https://towardsdatascience.com/divide-hugging-face-transformers-training-time-by-2-or-more-21bf7129db9q-21bf7129db9e

# 1 Preparação do ambiente
Preparação do ambiente para execução do notebook.

## 1.1 Tempo inicial de processamento

In [ ]:
import time
import datetime

# Marca o tempo de início do processamento
inicioProcessamento = time.time()

In [ ]:
print("  Tempo de início de processamento:  {:} (h:mm:ss)".format(inicioProcessamento))

  Tempo de início de processamento:  1623158860.356765 (h:mm:ss)


## 1.2 Funções e classes auxiliares

Função auxiliar para formatar o tempo como `hh: mm: ss`

In [ ]:
import time
import datetime

def formataTempo(tempo):
    '''
    Pega a tempo em segundos e retorna uma string hh:mm:ss
    '''
    # Arredonda para o segundo mais próximo.
    tempoArredondado = int(round((tempo)))
    
    # Formata como hh:mm:ss
    return str(datetime.timedelta(seconds=tempoArredondado))

Calcula a média de uma lista tempo string no formato hh:mm:ss.

In [ ]:
# Import das bibliotecas.
from cmath import rect, phase
from math import radians, degrees
  
def mediaAngulo(deg):
    return degrees(phase(sum(rect(1, radians(d)) for d in deg)/len(deg)))
 
def mediaTempo(tempos):
    '''
    Calcula a média de uma lista de tempo string no formato hh:mm:ss
    '''
    t = (tempo.split(':') for tempo in tempos)
    # Converte para segundos
    segundos = ((float(s) + int(m) * 60 + int(h) * 3600) for h, m, s in t)
    # Verifica se deu algum dia
    dia = 24 * 60 * 60
    # Converte para angulos
    paraAngulos = [s * 360. / dia for s in segundos]
    # Calcula a média dos angulos
    mediaComoAngulo = mediaAngulo(paraAngulos)
    media_segundos = mediaComoAngulo * dia / 360.
    if media_segundos < 0:
        media_segundos += dia
    # Recupera as horas e os minutos  
    h, m = divmod(media_segundos, 3600)
    # Recupera os minutos e os segundos
    m, s = divmod(m, 60)    
    return '{:02d}:{:02d}:{:02d}'.format(int(h), int(m), int(s))

Calcula a soma de uma lista de tempo string no formato hh:mm:ss

In [ ]:
def somaTempo(tempos):
    '''
    Calcula a soma de uma lista de tempo string no formato hh:mm:ss
    '''
    t = (tempo.split(':') for tempo in tempos)
    # Converte para segundos
    segundos = ((float(s) + int(m) * 60 + int(h) * 3600) for h, m, s in t)
    # Soma os segundos
    soma_segundos = sum([s * 1. for s in segundos])
    # Recupera as horas e os minutos   
    h, m = divmod(soma_segundos, 3600)
    # Recupera os minutos e os segundos
    m, s = divmod(m, 60)    
    return '{:02d}:{:02d}:{:02d}'.format(int(h), int(m), int(s))

Em muitos dos meus loops for (de longa duração), imprimirei atualizações periódicas de progresso. Normalmente, eu escolho o intervalo de atualização manualmente, mas para este Notebook, defini uma função auxiliar para fazer essa escolha para mim :)

In [ ]:
def obter_intervalo_atualizacao(total_iteracoes, numero_atualizacoes):
    '''
     Esta função tentará escolher um intervalo de atualização de progresso inteligente
     com base na magnitude das iterações totais.

     Parâmetros:
       `total_iteracoes` - O número de iterações no loop for.
       `numero_atualizacoes` - Quantas vezes queremos ver uma atualização sobre o
                               curso do loop for.
     '''
    
    # Divida o total de iterações pelo número desejado de atualizações. Provavelmente
    # este será um número feio.
    intervalo_exato = total_iteracoes / numero_atualizacoes

    # A função `arredondar` tem a capacidade de arredondar um número para, por exemplo, o
    # milésimo mais próximo: round (intervalo_exato, -3)
    #
    # Para determinar a magnitude para arredondar, encontre a magnitude do total,
    # e então vá uma magnitude abaixo disso.
    
    # Obtenha a ordem de magnitude do total.
    ordem_magnitude = len(str(total_iteracoes)) - 1
    
    # Nosso intervalo de atualização deve ser arredondado para uma ordem de magnitude menor.
    magnitude_arrendonda = ordem_magnitude - 1

    # Arredonde para baixo e lance para um int.
    intervalo_atualizacao = int(round(intervalo_exato, -magnitude_arrendonda))

    # Não permita que o intervalo seja zero!
    if intervalo_atualizacao == 0:
        intervalo_atualizacao = 1

    return intervalo_atualizacao

Classe(ModelArguments) de definição dos parâmetros do modelo

In [ ]:
# Import das bibliotecas.
from dataclasses import dataclass, field
from typing import Dict, Optional
from typing import List

@dataclass
class ModelArguments:
    max_seq_len: Optional[int] = field(
        default=None,
        metadata={"help": "max seq len"},
    )    
    pretrained_model_name_or_path: str = field(
        default="neuralmind/bert-base-portuguese-cased",
        metadata={"help": "nome do modelo pré-treinado do BERT."},
    )
    do_lower_case: bool = field(
        default=False,
        metadata={"help": "define se o texto do modelo deve ser todo em minúsculo."},
    )
    num_labels: int = field(
        default=2,
        metadata={"help": "número de rótulos a ser classificado."},
    )
    output_attentions: bool = field(
        default=False,
        metadata={"help": "habilita se o modelo retorna os pesos de atenção."},
    )
    output_hidden_states: bool = field(
        default=False,
        metadata={"help": "habilita gerar as camadas ocultas do modelo."},
    )
    optimizer: str = field(
        default="AdamW",
        metadata={"help": "otimizador do modelo."},
    )
    use_wandb : bool = field(
        default=True,
        metadata={"help": "habilita o uso do wandb."},
    )
    salvar_modelo_wandb : bool = field(
        default=True,
        metadata={"help": "habilita o salvamento do modelo no wandb."},
    )
    salvar_modelo : bool = field(
        default=True,
        metadata={"help": "habilita o salvamento do modelo."},
    )
    salvar_classificacao : bool = field(
        default=False,
        metadata={"help": "habilita o salvamento da classificação."},
    )
    salvar_avaliacao : bool = field(
        default=True,
        metadata={"help": "habilita o salvamento do resultado da avaliação."},
    )  
    

Funções auxiliares de arquivos

In [ ]:
def carregar(nomeArq):
    arq = open(nomeArq, 'r')
    
    paragrafo = ""
    for linha in arq:
        paragrafo = paragrafo + linha
    arq.close()
    return paragrafo

def carregarLista(nomeArq):
    arq = open(nomeArq, 'r')    
    seg = []
    for linha in arq:
      linha = linha.rstrip('\n')
      seg.append(linha)
    arq.close()
    return seg

def salvar(nomeArq,seg):                       
    arq = open(nomeArq, 'w')
    arq.write(str(seg))
    arq.close()

## 1.3 Tratamento de logs

In [ ]:
# Biblioteca de logging
import logging

# Formato da mensagem
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

## 1.4 Identificando o ambiente Colab

In [ ]:
# Se estiver executando no Google Colaboratory
import sys

# Retorna true ou false se estiver no Google Colaboratory
IN_COLAB = 'google.colab' in sys.modules

## 1.5 Biblioteca de limpeza de tela

In [ ]:
from IPython.display import clear_output

## 1.6 Conecta ao Google Drive

É necessário existir a pasta '/content/drive/MyDrive/Colab Notebooks/Data/CSTNEWS_MD_CV_10/Resultados/' para receber os resutlados do notebook.

In [ ]:
# Monta o Google Drive para esta instância de notebook.
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


## 1.7 Instalação do wandb

Instalação

In [ ]:
!pip install --upgrade wandb

     |████████████████████████████████| 1.8MB 2.9MB/s 
     |████████████████████████████████| 102kB 8.6MB/s 
     |████████████████████████████████| 133kB 20.4MB/s 
     |████████████████████████████████| 174kB 18.8MB/s 
     |████████████████████████████████| 71kB 7.6MB/s 
  Created wheel for subprocess32: filename=subprocess32-3.5.4-cp37-none-any.whl size=6502 sha256=406344acf7dd662a886b7cb403393295f67b22bcda41ddfbfa6129b67b845cbd
  Stored in directory: /root/.cache/pip/wheels/68/39/1a/5e402bdfdf004af1786c8b853fd92f8c4a04f22aad179654d1
  Created wheel for pathtools: filename=pathtools-0.1.2-cp37-none-any.whl size=8807 sha256=a0c0a4bb8851d665f3189d1a2b2c53c753a90150bf15b6c243652baf203f5cad
  Stored in directory: /root/.cache/pip/wheels/0b/04/79/c3b0c3a0266a3cb4376da31e5bfe8bba0c489246968a68e843
Successfully built subprocess32 pathtools


Login via linha de comando

In [ ]:
!wandb login aded3bc0ea651fff536cc08ba69caf8ac4141cfd

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


## 1.8 Instalação BERT da Hugging Face

Instala a interface pytorch para o BERT by Hugging Face. 

In [ ]:
!pip install -U transformers==4.5.1

     |████████████████████████████████| 2.1MB 2.8MB/s 
     |████████████████████████████████| 901kB 19.8MB/s 
     |████████████████████████████████| 3.3MB 24.1MB/s 


## 1.9 Colaboratory

Usando Colab GPU para Treinamento


Uma GPU pode ser adicionada acessando o menu e selecionando:

`Edit -> Notebook Settings -> Hardware accelerator -> (GPU)`

Em seguida, execute a célula a seguir para confirmar que a GPU foi detectada.

In [ ]:
# Importando a biblioteca
import tensorflow as tf

# Recupera o nome do dispositido da GPU.
device_name = tf.test.gpu_device_name()

# O nome do dispositivo deve ser parecido com o seguinte:
if device_name == '/device:GPU:0':
    print('Encontrei GPU em: {}'.format(device_name))
else:
    print('Dispositivo GPU não encontrado')
    #raise SystemError('Dispositivo GPU não encontrado')

Encontrei GPU em: /device:GPU:0


Nome da GPU

Para que a torch use a GPU, precisamos identificar e especificar a GPU como o dispositivo. Posteriormente, em nosso ciclo de treinamento, carregaremos dados no dispositivo.

Vale a pena observar qual GPU você recebeu. A GPU Tesla V100 é muito mais rápido que as outras GPUs, abaixo uma lista ordenada:
- 1o Tesla V100-SXM2-16GB(Pro)
- 2o Tesla P100-PCIE-16GB
- 3o Tesla T4
- 4o Tesla P4 (Não tem memória para execução 4 lotes de treino x 8 lotes de avaliação, somente 2 x 4)
- 5o Tesla K80 (Não tem memória para execução 4 lotes de treino x 8 lotes de avaliação, somente 2 x 4)

In [ ]:
# Importando a biblioteca
import torch

# Se existe GPU disponível...
if torch.cuda.is_available():    

    # Diz ao PyTorch para usar GPU.    
    device = torch.device("cuda")

    print('Existem {} GPU(s) disponíveis.'.format(torch.cuda.device_count()))

    print('Iremos usar a GPU: {}'.format(torch.cuda.get_device_name(0)))

# Se não...
else:
    print('Sem GPU disponível, usando CPU.')
    device = torch.device("cpu")

Existem 1 GPU(s) disponíveis.
Iremos usar a GPU: Tesla P100-PCIE-16GB


Memória

Memória disponível no ambiente

In [ ]:
# Importando as bibliotecas.
from psutil import virtual_memory

ram_gb = virtual_memory().total / 1e9
print('Seu ambiente de execução tem {: .1f} gigabytes de RAM disponível\n'.format(ram_gb))

if ram_gb < 20:
  print('Para habilitar um tempo de execução de RAM alta, selecione menu o ambiente de execução> "Alterar tipo de tempo de execução"')
  print('e selecione High-RAM. Então, execute novamente está célula')
else:
  print('Você está usando um ambiente de execução de memória RAM alta!')

Seu ambiente de execução tem  13.6 gigabytes de RAM disponível

Para habilitar um tempo de execução de RAM alta, selecione menu o ambiente de execução> "Alterar tipo de tempo de execução"
e selecione High-RAM. Então, execute novamente está célula


# 2 Parametrização

In [ ]:
# Importando as bibliotecas.
from transformers import TrainingArguments

# Definição dos parâmetros de Treinamento
training_args = TrainingArguments(
    # AjusteFinoMoodle_v1_C_SB_HT = nome do notebook
    # E = número de épocas
    # lr = taxa de aprendizagem
    # b = lotes de treino e avaliação    
    output_dir = 'AjusteFinoCSTNews_AvaliacaoMoodle_v1_C_SB_E_4_lr_1_b_4_8',  
    save_steps = 0,    
    seed = 42,
    num_train_epochs = 4, # Intervalo de valores: 2, 3, 4
    learning_rate = 1e-5, # Intervalo de valores: 1e-5, 2e-5, 3e-5, 4e-5, 5e-5 
    gradient_accumulation_steps = 1,
    per_device_train_batch_size = 4, 
    per_device_eval_batch_size = 8,        
    evaluation_strategy = 'epoch'
)

# Definição dos parâmetros do Modelo
model_args = ModelArguments(     
    max_seq_len = 512,
    #pretrained_model_name_or_path = "https://neuralmind-ai.s3.us-east-2.amazonaws.com/nlp/bert-large-portuguese-cased/bert-large-portuguese-cased_pytorch_checkpoint.zip",
    pretrained_model_name_or_path = "https://neuralmind-ai.s3.us-east-2.amazonaws.com/nlp/bert-base-portuguese-cased/bert-base-portuguese-cased_pytorch_checkpoint.zip",    
    #pretrained_model_name_or_path = 'bert-base-multilingual-cased',
    do_lower_case = False,   # default True
    num_labels = 2,
    output_attentions = False,    # default False
    output_hidden_states = False, # default False
    optimizer = 'AdamW',
    use_wandb = False,
    salvar_modelo_wandb = False,    
    salvar_modelo = False,
    salvar_classificacao = False, # Salva o resultado classificações
    salvar_avaliacao = False # Salva o resultado da avaliação das classificações
)

In [ ]:
# Verifica o nome do modelo BERT a ser utilizado
MODELO_BERT = 'SEM_MODELO_BERT'
if 'neuralmind' in model_args.pretrained_model_name_or_path:
  MODELO_BERT = '_BERTimbau'
else:
  if 'multilingual' in model_args.pretrained_model_name_or_path:
    MODELO_BERT = '_BERTmultilingual'

# Verifica o tamanho do modelo(default large)
TAMANHO_BERT = '_large'
if 'base' in model_args.pretrained_model_name_or_path:
  TAMANHO_BERT = '_base'

# 3 BERT

## 3.1 Arquivo do PyTorch Checkpoint

Lista de modelos da comunidade:
* https://huggingface.co/models

Português(https://github.com/neuralmind-ai/portuguese-bert):  
* **'neuralmind/bert-base-portuguese-cased'**
* **'neuralmind/bert-large-portuguese-cased'**

### Função download modelo

In [ ]:
def downloadModelo(MODELO):

  # Importando as bibliotecas.
  import os

  # Variável para setar o arquivo.
  URL_MODELO = None

  if 'http' in MODELO:
    URL_MODELO = MODELO

  # Se a variável foi setada.
  if URL_MODELO:

    # Diretório descompactação.
    DIRETORIO_MODELO = '/content/modelo'

    # Recupera o nome do arquivo do modelo da url.
    arquivo = URL_MODELO.split("/")[-1]

    # Nome do arquivo do vocabulário.
    arquivo_vocab = "vocab.txt"

    # Caminho do arquivo na url.
    caminho = URL_MODELO[0:len(URL_MODELO)-len(arquivo)]

    # Verifica se a pasta de descompactação existe na pasta corrente
    if os.path.exists(DIRETORIO_MODELO):
      print("Apagando diretório existente do modelo!")
      # Apaga a pasta e os arquivos existentes
      !rm -rf $DIRETORIO_MODELO  

    # Baixa o arquivo do modelo.
    !wget $URL_MODELO
    
    # Descompacta o arquivo na pasta de descompactação.
    !unzip -o $arquivo -d $DIRETORIO_MODELO

    # Baixa o arquivo do vocabulário.
    # O vocabulário não está no arquivo compactado acima, mesma url mas arquivo diferente.
    URL_MODELO_VOCAB = caminho + arquivo_vocab
    !wget $URL_MODELO_VOCAB
    
    # Coloca o arquivo do vocabulário no diretório de descompactação.
    !mv $arquivo_vocab $DIRETORIO_MODELO
            
    # Move o arquivo para pasta de descompactação
    !mv $arquivo $DIRETORIO_MODELO
       
    print('Pasta do {} pronta!'.format(DIRETORIO_MODELO))

    # Lista a pasta corrente.
    !ls -la $DIRETORIO_MODELO
  else:
    print('Variável URL_MODELO não setada!')

### Download do modelo

In [ ]:
downloadModelo(model_args.pretrained_model_name_or_path)

--2021-06-08 13:28:44--  https://neuralmind-ai.s3.us-east-2.amazonaws.com/nlp/bert-base-portuguese-cased/bert-base-portuguese-cased_pytorch_checkpoint.zip
Resolving neuralmind-ai.s3.us-east-2.amazonaws.com (neuralmind-ai.s3.us-east-2.amazonaws.com)... 52.219.102.82
Connecting to neuralmind-ai.s3.us-east-2.amazonaws.com (neuralmind-ai.s3.us-east-2.amazonaws.com)|52.219.102.82|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 406220891 (387M) [application/zip]
Saving to: ‘bert-base-portuguese-cased_pytorch_checkpoint.zip’

bert-base-portugues 100%[===================>] 387.40M  15.8MB/s    in 26s     

2021-06-08 13:29:11 (14.8 MB/s) - ‘bert-base-portuguese-cased_pytorch_checkpoint.zip’ saved [406220891/406220891]

Archive:  bert-base-portuguese-cased_pytorch_checkpoint.zip
  inflating: /content/modelo/config.json  
  inflating: /content/modelo/pytorch_model.bin  
--2021-06-08 13:29:16--  https://neuralmind-ai.s3.us-east-2.amazonaws.com/nlp/bert-base-portuguese-ca

## 3.2 Tokenizador(tokenizer) BERT

O tokenizador utiliza WordPiece, veja em [artigo original](https://arxiv.org/pdf/1609.08144.pdf).

Carregando o tokenizador da pasta '/content/modelo/' do diretório padrão se variável `URL_MODELO` setada.

**Caso contrário carrega da comunidade**

Por default(`do_lower_case=True`) todas as letras são colocadas para minúsculas. Para ignorar a conversão para minúsculo use o parâmetro `do_lower_case=False`. Esta opção também considera as letras acentuadas(ãçéí...), que são necessárias a língua portuguesa.

O parâmetro `do_lower_case` interfere na quantidade tokens a ser gerado apartir de um texto. Quando igual a `False` reduz a quantidade de tokens gerados.

### Função carrega tokenizador

In [ ]:
def carregaTokenizador(MODELO):

  # Importando as bibliotecas do tokenizador.
  from transformers import BertTokenizer

  # Variável para setar o arquivo.
  URL_MODELO = None

  if 'http' in MODELO:
    URL_MODELO = MODELO

  # Se a variável URL_MODELO foi setada.
  if URL_MODELO:

    # Diretório descompactação.
    DIRETORIO_MODELO = '/content/modelo'
    
    # Carregando o Tokenizador.
    print('Carregando o tokenizador BERT do diretório {}...'.format(DIRETORIO_MODELO))

    tokenizer = BertTokenizer.from_pretrained(DIRETORIO_MODELO, 
                                              do_lower_case=model_args.do_lower_case)
    
  else:
    # Carregando o Tokenizador da comunidade.
    print('Carregando o tokenizador da comunidade...')
    
    tokenizer = BertTokenizer.from_pretrained(MODELO, 
                                              do_lower_case=model_args.do_lower_case)

  return tokenizer

### Carregando o tokenizador

In [ ]:
tokenizer = carregaTokenizador(model_args.pretrained_model_name_or_path)

Carregando o tokenizador BERT do diretório /content/modelo...


## 3.3 Modelo(model) BERT

Se a variável `URL_MODELO` estiver setada carrega o modelo do diretório `content/modelo`.

Caso contrário carrega da comunidade.

Carregando o modelo da pasta '/content/modelo/' do diretório padrão.

A implementação do huggingface pytorch inclui um conjunto de interfaces projetadas para uma variedade de tarefas de PNL. Embora essas interfaces sejam todas construídas sobre um modelo treinado de BERT, cada uma possui diferentes camadas superiores e tipos de saída projetados para acomodar suas tarefas específicas de PNL.

A documentação para estas pode ser encontrada em [aqui](https://huggingface.co/transformers/v2.2.0/model_doc/bert.html).

Por default o modelo está em modo avaliação ou seja `model.eval()`.

-----------------------

Durante a avaliação do modelo, este retorna um número de diferentes objetos com base em como é configurado na chamada do método `from_pretrained`. 

Quando definimos `output_hidden_states = True` na chamada do método `from_pretrained`, retorno do modelo possui no terceiro item os estados ocultos(**hidden_states**) de todas as camadas.  Veja a documentação para mais detalhes: https://huggingface.co/transformers/model_doc/bert.html#bertmodel

Quando **`output_hidden_states = True`** model retorna:
- outputs[0] = last_hidden_state;
- outputs[1] = pooler_output; 
- outputs[2] = hidden_states.

Quando **`output_hidden_states = False`** ou não especificado model retorna:
- outputs[0] = last_hidden_state;
- outputs[1] = pooler_output.


**ATENÇÃO**: O parâmetro ´**output_hidden_states = True**´ habilita gerar as camadas ocultas do modelo. Caso contrário somente a última camada é mantida. Este parâmetro otimiza a memória mas não os resultados.


### Função carrega modelo

In [ ]:
def carregaModelo(MODELO):

  # Importando as bibliotecas do Modelo
  from transformers import BertForSequenceClassification

  # Variável para setar o arquivo.
  URL_MODELO = None

  if 'http' in MODELO:
    URL_MODELO = MODELO

  # Se a variável URL_MODELO foi setada
  if URL_MODELO:

    # Diretório descompactação.
    DIRETORIO_MODELO = '/content/modelo'
    
    # Carregando o Modelo BERT
    print('Carregando o modelo BERT do diretório {}...'.format(DIRETORIO_MODELO))

    model = BertForSequenceClassification.from_pretrained(DIRETORIO_MODELO,   
                                                          num_labels = model_args.num_labels,
                                                          output_attentions = model_args.output_attentions,
                                                          output_hidden_states = model_args.output_hidden_states)
  else:
    # Carregando o Modelo BERT da comunidade
    print('Carregando o modelo BERT da comunidade ...')
    
    model = BertForSequenceClassification.from_pretrained(MODELO,
                                                          num_labels = model_args.num_labels,                                                       
                                                          output_attentions = model_args.output_attentions,
                                                          output_hidden_states = model_args.output_hidden_states)
  return model

### Carregando o modelo

In [ ]:
model = carregaModelo(model_args.pretrained_model_name_or_path)

Carregando o modelo BERT do diretório /content/modelo...


Some weights of the model checkpoint at /content/modelo were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at /

# 4 Treino

## 4.1 wandb

https://wandb.ai/osmar-braz/ajustefinocstnews_v1_c_sb_holdout/table?workspace=user-osmar-braz

### Função de inicialização wandb

In [ ]:
def inicializacaoWandb():

  if model_args.use_wandb:

    # Importando a biblioteca.
    import wandb

    # Inicializando o registro do experimento.
    # Na execução só pode existir de um init  para que não gere dois registros no wandb.
    wandb.init(project="ajustefinocstnews_avaliacaomoodle_v1_c_sb", name=training_args.output_dir)

    # Atualiza os parâmetros de treinamento no wandb.
    wandb.config.update(training_args)
    # Atualiza os parâmetros do modelo no wandb.
    wandb.config.update(model_args)

    # Registra os parämetros não literais do model_args.
    wandb.log({"max_seq_len": model_args.max_seq_len})
    wandb.log({"do_lower_case": model_args.do_lower_case})
    wandb.log({"output_hidden_states": model_args.output_hidden_states})

    return wandb

### Inicialização wandb



In [ ]:
wandb = inicializacaoWandb()

## 4.2 Colab GPU

Conecta o modelo carregado do BERT a GPU para reduzir o tempo de processamento.

### Função conecta GPU

In [ ]:
def conectaGPU(model):
  
  # Associa a GPU ao modelo.
  model.to(device)

  # Se existe GPU disponível.
  if torch.cuda.is_available():    
    # Diga ao pytorch para rodar este modelo na GPU.
    print("Pytorch rodando o modelo na GPU")
    model.cuda()
  else:
    print("Pytorch rodando sem GPU")

  return model

### Conectando GPU ao modelo

In [ ]:
model = conectaGPU(model)

Pytorch rodando o modelo na GPU


## 4.3 Arquivo dos dados de treino

### função de download dos arquivos de dados

Download do arquivo dos dados de uma pasta pública no meu OneDrive.

In [ ]:
def downloadArquivoDados():
  
  # Nome do arquivo a ser criado.
  NOME_ARQUIVO = "Summarycoherencemodels.zip"

  # Apaga o arquivo.
  !rm '$NOME_ARQUIVO'

  # Realiza o download do arquivo da url especificada
  !wget https://sites.icmc.usp.br/taspardo/Summary%20coherence%20models.zip

  # Lista o diretório corrente e os arquivos.
  !pwd
  !ls -la

  # Descompactando os arquivos
  # Lista o diretório corrente e os arquivos.
  !pwd
  !ls -la

  # Apaga o diretório e seus arquivos
  !rm -rf "Summary coherence model"

  # Descompacta o arquivo
  !unzip -o '$NOME_ARQUIVO'

  # Lista os arquivos do diretório corrente
  !ls -la

  # Descompactando os experimentos
  NOME_ARQUIVO_EXPERIMENTO = 'Modelo Latent Semantic Analysis.zip'

  # Lista o diretório corrente e os arquivos.
  !pwd
  !ls -la

  # Apaga o diretório 'Modelo Latent Semantic Analysis' e seus arquivos
  !rm -rf 'Modelo Latent Semantic Analysis'

  # Descompacta o arquivo o experimento
  !unzip -o '$NOME_ARQUIVO_EXPERIMENTO'

  # Lista os arquivos do diretório corrente
  !ls -la

### Executando o download do arquivo de dados

In [ ]:
downloadArquivoDados()

rm: cannot remove 'Summarycoherencemodels.zip': No such file or directory
--2021-07-12 19:53:12--  https://sites.icmc.usp.br/taspardo/Summary%20coherence%20models.zip
Resolving sites.icmc.usp.br (sites.icmc.usp.br)... 143.107.183.230
Connecting to sites.icmc.usp.br (sites.icmc.usp.br)|143.107.183.230|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 498024248 (475M) [application/zip]
Saving to: ‘Summary coherence models.zip’

s.zip                 7%[>                   ]  34.54M   144KB/s    eta 82m 44s^C
/content
total 35424
drwxr-xr-x 1 root root     4096 Jul 12 19:53  .
drwxr-xr-x 1 root root     4096 Jul 12 19:46  ..
drwxr-xr-x 4 root root     4096 Jul  1 13:41  .config
drwxr-xr-x 1 root root     4096 Jul  1 13:42  sample_data
-rw-r--r-- 1 root root 36257792 Jul 12 19:59 'Summary coherence models.zip'
/content
total 35424
drwxr-xr-x 1 root root     4096 Jul 12 19:53  .
drwxr-xr-x 1 root root     4096 Jul 12 19:46  ..
drwxr-xr-x 4 root root     4096 Jul  1 1

## 4.4 Preparação dos dados Treino

### Função carregamento dados

Carrega os dados dos arquivos e uma lista e converte em um dataframe

Atributos do dataframe:
0. "idOriginal" - Nome do arquivo original
1. "sentencasOriginais" - Lista das sentenças do documento original
2. "documentoOriginal" - Documento original
3. "idPermutado" - Nome do arquivo permutado
4. "sentencasPermutadas" - Lista das sentenças do documento permtuado
5. "documentoPermutado" - Documento permutado

In [ ]:
def carregamentoDados():

  # Import das bibliotecas
  import os

  ############################################################
  # ORIGINAIS
  ############################################################

  lista_documentos_originais = []

  arquivos = os.listdir('/content/Modelo Latent Semantic Analysis/Sumarios_Humanos/') #Entrada (Input) - diretório de sumários humanos e permutados

  if '.DS_Store' in arquivos:
    arquivos.remove('.DS_Store')

  for i in range(len(arquivos)):
    # Recupera a posição do ponto no nome do arquivo
    ponto = arquivos[i].find('.')
    # Recupera o nome do arquivo até a posição do ponto
    nomeArquivo = arquivos[i][:ponto]

    documento = carregar('/content/Modelo Latent Semantic Analysis/Sumarios_Humanos/'+arquivos[i])

    lista_documentos_originais.append([documento,1])
    
  print ('TERMINADO ORIGINAIS')

  # Import das bibliotecas
  import os

  ############################################################
  # PERMUTADOS
  ############################################################

  lista_documentos_permutados = []

  arquivos = os.listdir('/content/Modelo Latent Semantic Analysis/Sumarios_Humanos_Permutados/') #Entrada (Input) - diret�rio de sum�rios humanos e permutados

  if '.DS_Store' in arquivos:
    arquivos.remove('.DS_Store')

  for i in range(len(arquivos)):
    # Recupera a posição do ponto no nome do arquivo
    ponto = arquivos[i].find('.')
    # Recupera o nome do arquivo até a posição do ponto
    nomeArquivo = arquivos[i][:ponto]

    documento = carregar('/content/Modelo Latent Semantic Analysis/Sumarios_Humanos_Permutados/'+arquivos[i])

    lista_documentos_permutados.append([documento,0])
    
  print ('TERMINADO INCOERENTES')

  print(len(lista_documentos_originais))
  print(len(lista_documentos_permutados))

  # Gerando os pares de documentos originais e permutados

  # Lista dos documentos originais e permutados 
  lista_documentos = []

  arquivosOriginais = os.listdir('/content/Modelo Latent Semantic Analysis/Sumarios_Humanos/') #Entrada (Input) - diretório de sumários humanos e permutados

  #del x[0](Comentando, pois o arquivo ".DS_Store" não está no início da lista!).
  if '.DS_Store' in arquivosOriginais:
    arquivosOriginais.remove('.DS_Store')

  for i in range(len(arquivosOriginais)):

    # Recupera a posição do ponto no nome do arquivo.
    ponto = arquivosOriginais[i].find('.')
    # Recupera o nome do arquivo até a posição do ponto.
    arquivoOriginal = arquivosOriginais[i][:ponto]

    # Carrega o documento original.
    # Carrega como parágrafo
    documentoOriginal = carregar('/content/Modelo Latent Semantic Analysis/Sumarios_Humanos/'+arquivosOriginais[i])
    # Carrega uma lista das sentenças
    sentencasOriginais = carregarLista('/content/Modelo Latent Semantic Analysis/Sumarios_Humanos/'+arquivosOriginais[i])

    # Percorre as 20 permutações.
    for j in range(20):
        # Recupera o nome do arquivo permutado.
        arquivoPermutado = arquivoOriginal + '_Perm_'+str(j)+'.txt'

        # Carrega o arquivo permutado.
        documentoPermutado = carregar('/content/Modelo Latent Semantic Analysis/Sumarios_Humanos_Permutados/'+ arquivoPermutado)
        sentencasPermutadas = carregarLista('/content/Modelo Latent Semantic Analysis/Sumarios_Humanos_Permutados/'+ arquivoPermutado)

        # Adiciona o par original e sua versão permutada.
        lista_documentos.append([arquivosOriginais[i], sentencasOriginais, documentoOriginal, arquivoPermutado, sentencasPermutadas, documentoPermutado])

  print(len(lista_documentos))

  # Converte a lista em um dataframe

  # Import das bibliotecas.
  import pandas as pd

  # Converte a lista em um dataframe.
  dfdados = pd.DataFrame.from_records(lista_documentos, columns=['idOriginal','sentencasOriginais','documentoOriginal','idPermutado','sentencasPermutadas','documentoPermutado'])

  # Número de linhas carregadas do arquivo.
  print('Total de registros              : {}'.format(len(dfdados)))

  # Organiza os dados

  dados_organizados = []

  # Coloca o par um embaixo do outro.
  for index, linha in dfdados.iterrows():        
    # 1 Para original
    dados_organizados.append((linha['idOriginal'],linha['documentoOriginal'],1))    
    # 0 para uma permutação 
    dados_organizados.append((linha['idPermutado'],linha['documentoPermutado'],0))

  # Cria um dataframe com os dados
  dfdados = pd.DataFrame(dados_organizados, columns=["id","documento","classe"])      

  return dfdados 


### Carregamento dos dados

In [ ]:
dfdados = carregamentoDados()

TERMINADO ORIGINAIS
TERMINADO INCOERENTES
251
5020
5020
Total de registros              : 5020


### Função descarte documentos muito grandes

In [ ]:
def descarteDocumentosGrandes(tamanho_maximo_token, dfdados):
  
  # Define o tamanho máximo para os tokens.
  tamanho_maximo = tamanho_maximo_token

  # Tokenize a codifica as setenças para o BERT.     
  dfdados['input_ids'] = dfdados['documento'].apply(lambda tokens: tokenizer.encode(tokens, add_special_tokens=True))
        
  dfdados = dfdados[dfdados['input_ids'].apply(len)<tamanho_maximo]

  print('Tamanho do conjunto de dados: {}'.format(len(dfdados)))

  # Remove as colunas desnecessárias.
  dfdados = dfdados.drop(columns=['input_ids'])

  # Informações do DataFrame
  print(dfdados.info())

  return dfdados

### Descartando documentos muito grandes

In [ ]:
dfdados = descarteDocumentosGrandes(model_args.max_seq_len, dfdados)

2021-06-08 13:30:31,594 : INFO : NumExpr defaulting to 2 threads.


Tamanho do conjunto de dados: 9960
<class 'pandas.core.frame.DataFrame'>
Int64Index: 9960 entries, 0 to 10039
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         9960 non-null   object
 1   documento  9960 non-null   object
 2   classe     9960 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 311.2+ KB
None


### Divisão do conjunto de dados

Divide nosso conjunto de treinamento para usar 70% para treinamento e 30% para validação.

### Função divisão conjunto de dados

In [ ]:
def divisaoConjuntoDados(dfdados):
  
  # Import das bibliotecas.
  from sklearn.model_selection import train_test_split

  #30% de teste
  test_qtde = int(0.3*dfdados.shape[0])
  dfdados_train, dfdados_test = train_test_split(dfdados, test_size=test_qtde, random_state=42, stratify=dfdados['classe'])

  len(dfdados_train), len(dfdados_test)

  return dfdados_train, dfdados_test


### Divisão do conjunto de dados

In [ ]:
 #dfdados_train, dfdados_test = divisaoConjuntoDados(dfdados)
 dfdados_train = dfdados

Vamos extrair os dados do arquivo do TensorFlow, para termos apenas tipos simples de Python.

Não foi usada a classe tensorflow_datasets, portanto não foi necessária a extração, somente a divisão em listas separadas.

### Seleciona as colunas de treino

In [ ]:
# Import das bibliotecas.
import numpy as np

# Pega as listas de documentos e seus rótulos para o treino
documentos_treino = dfdados_train.documento.values
classes_treino = dfdados_train.classe.values
documentoids_treino = dfdados_train.id.values

# Mostra algumas estatísticas.
print('{:,} Amostras de Treino'.format(len(documentos_treino)))
print('{:,} Rótulos de Treino'.format(len(classes_treino)))
print('Rótulos: {}'.format(np.unique(classes_treino)))

9,960 Amostras de Treino
9,960 Rótulos de Treino
Rótulos: [0 1]


## 4.5 Análise treino

Usaremos os pandas para analisar o conjunto de dados e examinar algumas de suas propriedades e pontos de dados.

Atributos da lista:
0. "arquivo"
1. "documento"
2. "classe" (1-Original, 0-Permutado)


In [ ]:
dfdados_train.sample(5)

,id,documento,classe
9115,C41_Extrato_3_Perm_17.txt,Thiago Pereira foi mais uma vez a estrela da n...,0
9585,C3_Extrato_4_Perm_12.txt,"Para a TAM, a falha não impediria a realização...",0
86,C49_Extrato_4.txt,O presidente Luiz Inácio Lula da Silva classif...,1
10039,C46_Extrato_5_Perm_19.txt,Cerca de 1.700 pessoas fugiram de suas casas p...,0
3023,C36_Extrato_1_Perm_11.txt,Sobreviver politicamente era queestão de homra...,0


In [ ]:
# Mostra o número de documento de treino.
print('Número de documentos de treino: {:,}\n'.format(dfdados_train.shape[0]))

# Informações do DataFrame.
print(dfdados_train.info())

Número de documentos de treino: 9,960

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9960 entries, 0 to 10039
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         9960 non-null   object
 1   documento  9960 non-null   object
 2   classe     9960 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 311.2+ KB
None


### Distribuição das classes

O dataset está bem balanceado, o que nos conduz a utilizar acurácia como métrica.

In [ ]:
dfdados_train.groupby('classe').count()

,id,documento
classe,,
0,4980,4980
1,4980,4980


In [ ]:
# Informações do DataFrame.
print(dfdados_train.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9960 entries, 0 to 10039
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         9960 non-null   object
 1   documento  9960 non-null   object
 2   classe     9960 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 311.2+ KB
None


### Conjunto de dados em Treinamento

In [ ]:
# Mostra o resultado dos dados carregados.
print("Total do conjunto de dados          : {}.".format(len(dfdados)))
print("Total do conjunto de dados de treino: {}.".format(len(documentos_treino)))

Total do conjunto de dados          : 9960.
Total do conjunto de dados de treino: 9960.


## 4.6 Arquivo dos dados avaliacao

### Função de download dos arquivos de dados



In [ ]:
def downloadArquivoDados():
  
  # Especifica o nome do arquivo de dados
  # Nome do arquivo
  NOMEARQUIVOORIGINAL = 'original.zip'
  NOMEARQUIVOPERMUTADO = 'permutado.zip'

  # Especifica o caminho dos arquivos
  # Define o caminho e nome do arquivo de dados
  CAMINHOARQUIVOORIGINAL = '/content/drive/MyDrive/Colab Notebooks/Data/Moodle/dadosmoodle_documento_pergunta_sentenca_intervalo/' + NOMEARQUIVOORIGINAL
  CAMINHOARQUIVOPERMUTADO = '/content/drive/MyDrive/Colab Notebooks/Data/Moodle/dadosmoodle_documento_pergunta_sentenca_intervalo/' + NOMEARQUIVOPERMUTADO
  
  # Copia os arquivos do Google Drive para o Colaboratory
  # Copia os arquivos dos dados originais e permutados par ao diretório corrente.

  !cp '/content/drive/MyDrive/Colab Notebooks/Data/Moodle/dadosmoodle_documento_pergunta_sentenca_intervalo/'$NOMEARQUIVOORIGINAL .
  !cp '/content/drive/MyDrive/Colab Notebooks/Data/Moodle/dadosmoodle_documento_pergunta_sentenca_intervalo/'$NOMEARQUIVOPERMUTADO .

  print("Terminei a cópia!")

  # Descompacta os arquivos
  !unzip -o -q $NOMEARQUIVOORIGINAL
  !unzip -o -q $NOMEARQUIVOPERMUTADO

  print("Terminei a descompactação!")

### Executando o download do arquivo de dados

In [ ]:
downloadArquivoDados()

Terminei a cópia!
Terminei a descompactação!


## 4.7 Preparação dos dados avaliação

### Função carregamento dados

Carrega os dados dos arquivos e uma lista e converte em um dataframe

Atributos do dataframe:
0. "idOriginal" - Nome do arquivo original
1. "sentencasOriginais" - Lista das sentenças do documento original
2. "documentoOriginal" - Documento original
3. "idPermutado" - Nome do arquivo permutado
4. "sentencasPermutadas" - Lista das sentenças do documento permtuado
5. "documentoPermutado" - Documento permutado

In [ ]:
def carregamentoDados():

  # Biblioteca para acessar o sistema de arquivos
  import os

  ############################################################
  # Original
  ############################################################

  lista_documentos_originais = []

  arquivos = os.listdir('/content/dadosmoodle_documento_pergunta_sentenca_intervalo/original/') #Entrada (Input)

  # Percorre a lista de arquivos do diretório
  for i in range(len(arquivos)):
    # Recupera a posição do ponto no nome do arquivo
    ponto = arquivos[i].find('.')
    # Recupera o nome do arquivo até a posição do ponto
    nomeArquivo = arquivos[i][:ponto]

    # Carrega o arquivo de nome x[i] do diretório
    documento = carregarLista('/content/dadosmoodle_documento_pergunta_sentenca_intervalo/original/'+arquivos[i])

    lista_documentos_originais.append([nomeArquivo,documento])
    
  print ('TERMINADO ORIGINAL')
  print(len(lista_documentos_originais))

  # Biblioteca para acessar o sistema de arquivos
  import os

  ############################################################
  # Permutado
  ############################################################

  lista_documentos_permutados = []

  arquivos = os.listdir('/content/dadosmoodle_documento_pergunta_sentenca_intervalo/permutado/') #Entrada (Input)

  # Percorre a lista de arquivos do diretório
  for i in range(len(arquivos)):
    # Recupera a posição do ponto no nome do arquivo
    ponto = arquivos[i].find('.')
    # Recupera o nome do arquivo até a posição do ponto
    nomeArquivo = arquivos[i][:ponto]

    # Carrega o arquivo de nome x[i] do diretório
    documento = carregarLista('/content/dadosmoodle_documento_pergunta_sentenca_intervalo/permutado/'+arquivos[i])

    # Adiciona a lista o conteúdo do arquivo
    lista_documentos_permutados.append([nomeArquivo,documento])
    
  print ('TERMINADO ORIGINAL')
  print(len(lista_documentos_permutados))

  # Gerando os pares de documentos originais e permutados

  # Lista dos documentos originais e permutados unificados
  lista_documentos = []

  # Percorre a lista dos documentos originais
  for i,linha_original in enumerate(lista_documentos_originais):

    # Percorre a lista dos documentos permutados
    for j,linha_permutado in enumerate(lista_documentos_permutados):

        # Se o nome o id documento original está no nome do documento permutado 
        if linha_original[0] in linha_permutado[0]:

          # Adiciona os dados a lista dos documentos
          documentoOriginal = " ".join(linha_original[1])
          documentoPermutado = " ".join(linha_permutado[1])

          # Adiciona os dados originais e permutados a lista
          lista_documentos.append([linha_original[0],linha_original[1], documentoOriginal, linha_permutado[0],linha_permutado[1], documentoPermutado ])

  print(len(lista_documentos))

  # Converte a lista em um dataframe

  # Import das bibliotecas.
  import pandas as pd

  # Converte a lista em um dataframe.
  dfdados = pd.DataFrame.from_records(lista_documentos, columns=['idOriginal','sentencasOriginais','documentoOriginal','idPermutado','sentencasPermutadas','documentoPermutado'])

  # Número de linhas carregadas do arquivo.
  print('Total de registros              : {}'.format(len(dfdados)))

  # Organiza os dados
  
  dados_organizados = []

  # Coloca o par um embaixo do outro.
  for index, linha in dfdados.iterrows():        
    # 1 Para original
    dados_organizados.append((linha['idOriginal'],linha['documentoOriginal'],1))    
    # 0 para uma permutação 
    dados_organizados.append((linha['idPermutado'],linha['documentoPermutado'],0))

  # Cria um dataframe com os dados de teste
  dfdados = pd.DataFrame(dados_organizados, columns=["id","documento","classe"])      

  return dfdados 

### Carregamento dos dados

In [ ]:
dfdados = carregamentoDados()

TERMINADO ORIGINAL
561
TERMINADO ORIGINAL
11220
11220
Total de registros              : 11220


### Descartando os documentos muito grandes

### Função descarte documentos muito grandes

In [ ]:
def descarteDocumentosGrandes(tamanho_maximo_token, dfdados):
  
  # Define o tamanho máximo para os tokens.
  tamanho_maximo = tamanho_maximo_token

  # Tokenize a codifica as setenças para o BERT.     
  dfdados['input_ids'] = dfdados['documento'].apply(lambda tokens: tokenizer.encode(tokens, add_special_tokens=True))
        
  dfdados = dfdados[dfdados['input_ids'].apply(len)<tamanho_maximo]

  print('Tamanho do conjunto de dados: {}'.format(len(dfdados)))

  # Remove as colunas desnecessárias.
  dfdados = dfdados.drop(columns=['input_ids'])

  # Informações do DataFrame
  print(dfdados.info())

  return dfdados

### Descartando os documentos muito grandes

In [ ]:
dfdados = descarteDocumentosGrandes(model_args.max_seq_len, dfdados)

Tamanho do conjunto de dados: 22440
<class 'pandas.core.frame.DataFrame'>
Int64Index: 22440 entries, 0 to 22439
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         22440 non-null  object
 1   documento  22440 non-null  object
 2   classe     22440 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 701.2+ KB
None


### Divisão do conjunto de dados

Divide nosso conjunto de treinamento para usar 70% para treinamento e 30% para validação.

### Função divisão conjunto de dados

In [ ]:
def divisaoConjuntoDados(dfdados):
  
  # Import das bibliotecas.
  from sklearn.model_selection import train_test_split

  #30% de teste
  test_qtde = int(0.3*dfdados.shape[0])
  dfdados_train, dfdados_test = train_test_split(dfdados, test_size=test_qtde, random_state=42, stratify=dfdados['classe'])

  len(dfdados_train), len(dfdados_test)

  return dfdados_train, dfdados_test

### Divisão do conjunto de dados

In [ ]:
 #dfdados_train, dfdados_test = divisaoConjuntoDados(dfdados)
 dfdados_test = dfdados

Vamos extrair os dados do arquivo do TensorFlow, para termos apenas tipos simples de Python.

Não foi usada a classe tensorflow_datasets, portanto não foi necessária a extração, somente a divisão em listas separadas.

### Seleciona as colunas de teste

In [ ]:
# Import das bibliotecas.
import numpy as np

# Pega as listas de documentos e seus rótulos para o treino
documentos_teste = dfdados_test.documento.values
classes_teste = dfdados_test.classe.values
documentoids_teste = dfdados_test.id.values

# Mostra algumas estatísticas.
print('{:,} Amostras de Teste'.format(len(documentos_teste)))
print('{:,} Rótulos de Teste'.format(len(classes_teste)))
print('Rótulos: {}'.format(np.unique(classes_teste)))

22,440 Amostras de Teste
22,440 Rótulos de Teste
Rótulos: [0 1]


## 4.8 Análise avaliação

Usaremos os pandas para analisar o conjunto de dados e examinar algumas de suas propriedades e pontos de dados.

Atributos da lista:
0. "arquivo"
1. "documento"
2. "classe" (1-Original, 0-Permutado)


In [ ]:
dfdados_test.sample(5)

,id,documento,classe
14157,Documento_74940_Perm_1,Ficou alguma dúvida quanto ao exercício e à pe...,0
5362,Documento_47613,"A tirinha acima nos mostra, a que a TV, como m...",1
10085,Documento_103572_Perm_6,Olá Simone tudo bem? No cadastro de cliente fa...,0
21301,Documento_68115_Perm_11,Elisa MannesTutora a distância Posso te ajudar...,0
15685,Documento_40930_Perm_13,"O que te faz realmente humano? A escola, com s...",0


In [ ]:
# Mostra o número de documento de treino.
print('Número de documentos de teste: {:,}\n'.format(dfdados_test.shape[0]))

# Informações do DataFrame.
print(dfdados_test.info())

Número de documentos de teste: 22,440

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22440 entries, 0 to 22439
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         22440 non-null  object
 1   documento  22440 non-null  object
 2   classe     22440 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 701.2+ KB
None


### Distribuição das classes

O dataset está bem balanceado, o que nos conduz a utilizar acurácia como métrica.

In [ ]:
dfdados_test.groupby('classe').count()

,id,documento
classe,,
0,11220,11220
1,11220,11220


In [ ]:
# Informações do DataFrame.
print(dfdados_test.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22440 entries, 0 to 22439
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         22440 non-null  object
 1   documento  22440 non-null  object
 2   classe     22440 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 701.2+ KB
None


### Conjunto de dados em Avaliação

In [ ]:
# Mostra o resultado dos dados carregados.
print("Total do conjunto de dados          : {}.".format(len(dfdados)))
print("Total do conjunto de dados de teste : {}.".format(len(documentos_teste)))

Total do conjunto de dados          : 22440.
Total do conjunto de dados de teste : 22440.


## 4.6 Treinando o modelo de classificação

### Otimizador e Agendador de Taxas de Aprendizado/Optimizer & Learning Rate Scheduler



Agora que temos nosso modelo carregado, precisamos pegar os hiperparâmetros de treinamento no modelo armazenado.

Para fins de ajuste fino, os autores recomendam escolher entre os seguintes valores (no Apêndice A.3 do [artigo BERT](https://arxiv.org/pdf/1810.04805.pdf)):

> - **Tamanho do lote(Batch size):** 16, 32
- **Taxa de aprendizado (Adam):** 5e-5, 3e-5, 2e-5
- **Número de épocas:** 2, 3, 4

O parâmetro epsilon `eps = 1e-6` é" um número muito pequeno para impedir qualquer divisão por zero na implementação "(a partir de [aqui](https://machinelearningmastery.com/adam-optimization-algorithm-for-deep-learning/)).

Você pode encontrar a criação do otimizador do AdamW em `run_glue.py` [aqui](https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L109).

### Função carrega otimizador

In [ ]:
def carregaOtimizador():

  '''
    Esta função carrega o otimizador utilizado no agendador de aprendizado.
  '''
  
  # Import das bibliotecas.
  from transformers import AdamW

  # Nota: AdamW é uma classe da biblioteca huggingface (ao contrário de pytorch).
  # Eu acredito que o 'W' significa 'Correção de redução de peso "
  optimizer = AdamW(model.parameters(),
                  lr = training_args.learning_rate, # (ou alfa) A taxa de aprendizado a ser usada. - default é 3e-5
                  # betas = (0.9, 0.999), # (beta1, beta2) - default é (0.9, 0.999)
                    # beta1 é taxa de decaimento exponencial para as estimativas do primeiro momento. 
                    # beta2 é taxa de decaimento exponencial para as estimativas do segundo momento. Este valor deve ser definido próximo a 1,0 em problemas com gradiente esparso (por exemplo, PNL e problemas de visão de computacional)
                  # eps = 1e-6, #  É um número muito pequeno para evitar qualquer divisão por zero na implementação - default é 1e-6.
                  # weight_decay = 0.0, # Correção de redução de peso. - default é 0.0
                    # A redução da taxa de aprendizagem também pode ser usada com Adam. A taxa de decaimento é atualizada a cada época para a demonstração da regressão logística.
                  # correct_bias = True #  Se não deve corrigir o viés(bias) no Adam mudar para False.- default é True
                )
  
  return optimizer

### Carregando otimizador

In [ ]:
optimizer = carregaOtimizador()

### Função carrega agendador

A função **get_linear_schedule_with_warmup** cria um agendador com uma taxa de aprendizado que diminua linearmente da taxa de aprendizagem inicial definido no otimizador até 0, após um período de aquecimento durante o qual ele aumenta linearmente de 0 para a taxa de aprendizagem inicial definido no otimizador.

Se `num_warmup_steps=0` e `weight_decay=0`(otimizador) não ocorre a etapa de aquecimento.

In [ ]:
def carregaAgendador():

  '''
    Esta função carrega o agendador com um taxa de aprendizado que diminua linearmente até 0.
  '''

  # Import das bibliotecas.
  from transformers import get_linear_schedule_with_warmup

  # O número total de etapas de ajuste fino é [número de lotes] x [número de épocas].
  # (Observe que este não é o mesmo que o número de amostras de ajuste fino).
  total_etapas = len(documentos_treino) * training_args.num_train_epochs

  #Cria o agendador de taxa de aprendizagem.
  scheduler = get_linear_schedule_with_warmup(optimizer, # O otimizador para o qual agendar a taxa de aprendizado.
                                            num_warmup_steps = 0, # O número de etapas para a fase de aquecimento. Valor default value em run_glue.py
                                            num_training_steps = total_etapas) # O número total de etapas de treinamento.


  print("Total de etapas: {}".format(total_etapas))

  return scheduler

### Carrega agendador

In [ ]:
scheduler = carregaAgendador()

Total de etapas: 39840


### Função cria lotes inteligentes

In [ ]:
def cria_lotes_inteligentes(documentos, classes, documentoids, batch_size):
    '''
    Esta função combina todos os passos para preparar os lotes.
    '''
    print('Criando Lotes Inteligentes de {:,} amostras com tamanho de lote {:,}...\n'.format(len(documentos), batch_size))

    # ============================
    #   Tokenização & Truncamento
    # ============================

    input_ids_completos = []
    
    # Tokeniza todas as amostras de treinamento
    print('Tokenizando {:,} amostra...'.format(len(classes)))
    
    # Escolha o intervalo que o progresso será atualizado.
    intervalo_atualizacao = obter_intervalo_atualizacao(total_iteracoes=len(classes), numero_atualizacoes=10)
    
    # Para cada amostra de treinamento...
    for documento in documentos:
        
        # Relatório de progresso
        if ((len(input_ids_completos) % intervalo_atualizacao) == 0):
            print('  Tokenizado {:,} amostras.'.format(len(input_ids_completos)))

        # Tokeniza a amostra.
        input_ids = tokenizer.encode(text=documento,                    # Documento a ser codificado.
                                    add_special_tokens=True,            # Adiciona os ttokens especiais.
                                    max_length=model_args.max_seq_len,  # Tamanho do truncamento!
                                    truncation=True,                    # Faz o truncamento!
                                    padding=False)                      # Não preenche.
                
        # Adicione o resultado tokenizado à nossa lista.
        input_ids_completos.append(input_ids)
        
    print('FEITO.')
    print('{:>10,} amostras\n'.format(len(input_ids_completos)))

    # =========================
    #      Seleciona os Lotes
    # =========================    
    
    # Classifique as duas listas pelo comprimento da sequência de entrada.
    amostras = sorted(zip(input_ids_completos, classes, documentoids), key=lambda x: len(x[0]))

    print('{:>10,} amostras após classificação\n'.format(len(amostras)))

    import random

    # Lista de lotes que iremos construir.
    batch_ordered_documentos = []
    batch_ordered_classes = []
    batch_ordered_documentoids = []

    print('Criando lotes de tamanho {:}...'.format(batch_size))

    # Escolha um intervalo no qual imprimir atualizações de progresso.
    intervalo_atualizacao = obter_intervalo_atualizacao(total_iteracoes=len(amostras), numero_atualizacoes=10)
        
    # Faça um loop em todas as amostras de entrada ... 
    while len(amostras) > 0:
        
        # Mostra o progresso.
        if ((len(batch_ordered_documentos) % intervalo_atualizacao) == 0 \
            and not len(batch_ordered_documentos) == 0):
            print('  Selecionado {:,} lotes.'.format(len(batch_ordered_documentos)))
        
        # `to_take` é o tamanho real do nosso lote. Será `batch_size` até
        # chegamos ao último lote, que pode ser menor.
        to_take = min(batch_size, len(amostras))
        
        # Escolha um índice aleatório na lista de amostras restantes para começar o nosso lote.
        select = random.randint(0, len(amostras) - to_take)

        # Selecione um lote contíguo de amostras começando em `select`.
        #print ("Selecionando lote de {:} a {:}".format(select, select+to_take))
        batch = amostras[select:(select + to_take)]

        #print("Tamanho do lote:", len(batch))
        
        # Cada amostra é uma tupla --divida para criar uma lista separada de
        # sequências e uma lista de rótulos para este lote.
        batch_ordered_documentos.append([s[0] for s in batch])
        batch_ordered_classes.append([s[1] for s in batch])
        batch_ordered_documentoids.append([s[2] for s in batch])
        
        # Remova a amostra da lista
        del amostras[select:select + to_take]

    print('\n  FEITO - Selecionado {:,} lotes.\n'.format(len(batch_ordered_documentos)))

    # =========================
    #        Adicionando o preenchimento
    # =========================    

    print('Preenchendo sequências dentro de cada lote...')

    py_input_ids = []
    py_attention_masks = []
    py_labels = []
    list_documentoids = []

    # Para cada lote...
    for (batch_input_ids, batch_labels, batch_documentoids) in zip(batch_ordered_documentos, batch_ordered_classes, batch_ordered_documentoids):

        # Nova versão do lote, desta vez com sequências preenchidas e agora com
        # as máscaras de atenção definidas.
        batch_padded_input_ids = []
        batch_attention_masks = []
                
        # Primeiro, encontre a amostra mais longa do lote.
        # Observe que as sequências atualmente incluem os tokens especiais!
        max_size = max([len(input) for input in batch_input_ids])
        
        # Para cada entrada neste lote...
        for input in batch_input_ids:
                        
            # Quantos tokens pad precisam ser adicionados
            num_pads = max_size - len(input)

            # Adiciona `num_pads` do pad token(tokenizer.pad_token_id) até o final da sequência.
            padded_input = input + [tokenizer.pad_token_id] * num_pads

            # Define a máscara de atenção --é apenas um `1` para cada token real
            # e um `0` para cada token de preenchimento(pad).
            attention_mask = [1] * len(input) + [0] * num_pads
            
            # Adiciona o resultado preenchido ao lote.
            batch_padded_input_ids.append(padded_input)
            batch_attention_masks.append(attention_mask)
        
        # Nosso lote foi preenchido, portanto, precisamos salvar este lote atualizado.
        # Também precisamos que as entradas sejam tensores PyTorch, então faremos isso aqui.
        py_input_ids.append(torch.tensor(batch_padded_input_ids))
        py_attention_masks.append(torch.tensor(batch_attention_masks))
        py_labels.append(torch.tensor(batch_labels))
        list_documentoids.append(batch_documentoids)
    
    # Retorna o conjunto de dados em lotes inteligentes!
    return (py_input_ids, py_attention_masks, py_labels, list_documentoids)

### Função de Treinamento

In [ ]:
# Import das bibliotecas
import random
import numpy as np
from tqdm.notebook import tqdm as tqdm_notebook

def realizaTreinamento(documentos_treino, classes_treino, documentoids_treino, EPOCAS = 4):
  
  print("\nRealizando Treinamento ")

  # Defina o valor da semente em todos os lugares para torná-lo reproduzível.
  seed_val = training_args.seed

  random.seed(seed_val)
  np.random.seed(seed_val)
  torch.manual_seed(seed_val)
  torch.cuda.manual_seed_all(seed_val)

  # Atualize todos os lotes ʻintervalo_atualizacao`.
  intervalo_atualizacao = obter_intervalo_atualizacao(total_iteracoes=len(documentos_treino), numero_atualizacoes=10)

  # Medida do tempo total de treinamento.
  treinamento_t0 = time.time()

  # Limpa o cache da GPU.
  torch.cuda.empty_cache()

  # Coloque o modelo em modo de treinamento. 
  model.train()

  # Acumula as perdas do treinamento.
  train_losses = []

  if model_args.use_wandb:
    # Log das métricas com wandb.
    wandb.watch(model)

  # Barra de progresso da época.
  epoca_bar = tqdm_notebook(range(training_args.num_train_epochs), desc=f'Épocas', unit=f'épocas')

  # Para cada época.
  for epoca_i in epoca_bar:
    
    # ========================================
    #               Treinamento
    # ========================================
    
    # Execute uma passada completa sobre o conjunto de treinamento.

    # Recupera o lote inteligente
    (py_input_ids, py_attention_masks, py_labels, documentoids) = cria_lotes_inteligentes(documentos_treino, classes_treino, documentoids_treino, training_args.per_device_train_batch_size)

    # Medida de quanto tempo leva o período de treinamento.
    treinamento_epoca_t0 = time.time()

    # Acumula as perdas do treinamento da época.
    train_epoca_losses = []

    # Barras de progresso.    
    lote_treino_bar = tqdm_notebook(range(0, len(py_input_ids)), desc=f'Epoca {epoca_i+1}', unit=f'lotes', total=len(py_input_ids) )

    # Para cada lote dos dados de treinamento.
    for index in lote_treino_bar:      

        # Progresso é atualizado a cada lotes, por exemplo, 100 lotes.
        if index % intervalo_atualizacao == 0 and not index == 0:            
            # Calcula gasto o tempo em minutos.
            tempoGasto = formataTempo(time.time() - treinamento_epoca_t0)
                        
            # Calcule o tempo restante com base em nosso progresso.
            passos_por_segundo = (time.time() - treinamento_epoca_t0) / index
            segundos_restantes = passos_por_segundo * (len(py_input_ids) - index)
            tempoRestante = formataTempo(segundos_restantes)

            # Mostra o progresso.
            print('  Lote {:>7,}  de  {:>7,}.    Gasto: {:}.  Restante: {:}'.format(index, len(py_input_ids), tempoGasto, tempoRestante))

        # Descompacte este lote de treinamento de nosso dataloader.
        #
        # À medida que descompactamos o lote, também copiaremos cada tensor para a GPU usando o
        # o método `to`
        #
        # `lote` é uma lista contém três tensores pytorch:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 

        # Recupera os tensores do lote e copia para a GPU usando o método `to` 
        d_input_ids = py_input_ids[index].to(device)
        d_input_mask = py_attention_masks[index].to(device)
        d_labels = py_labels[index].to(device)     
        
        # Sempre limpe quaisquer gradientes calculados anteriormente antes de realizar um
        # passe para trás. PyTorch não faz isso automaticamente porque
        # acumular os gradientes é "conveniente durante o treinamento de RNNs".
        # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
        model.zero_grad()

        # Execute um passe para frente (avalie o modelo neste lote de treinamento).
        # A documentação para esta função `model` está aqui:
        # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
        # Ele retorna diferentes números de parâmetros dependendo de quais argumentos
        # são fornecidos e quais sinalizadores estão definidos. Para nosso uso aqui, ele retorna
        # a perda (porque fornecemos rótulos) e os "logits" - o modelo de saídas antes da ativação.     

        # last_hidden_state = outputs[0], pooler_output = outputs[1], hidden_states = outputs[2]
        outputs = model(d_input_ids, 
                        token_type_ids=None, 
                        attention_mask=d_input_mask, 
                        labels=d_labels)
        
        # A perda(loss) é retornado em outputs[0] porque fornecemos rótulos(labels))                  
        loss = outputs[0]

        # E outputs[1] os "logits" - o modelo de saídas antes da ativação.
        # logits possui duas dimensões, a primeira do lote e a segunda do rótulo da predição                        
        # A função `.detach().cpu()` retira da gpu.
        logits = outputs[1].detach().cpu()
  
        # Acumule a perda de treinamento em todos os lotes da época para que possamos
        # calcular a perda média no final da época. `loss` é um tensor contendo um único valor.   
        # A função '.cpu()' move loss para a cpu.
        # A função `.item ()` retorna apenas o valor Python do tensor.
        train_epoca_losses.append(loss.cpu().item())
        
        # Mostra a perda na barra de progresso.
        lote_treino_bar.set_postfix(loss=loss.cpu().item())

        if model_args.use_wandb:
          wandb.log({"train_batch_loss": loss.cpu().item()})

        # Execute uma passagem para trás para calcular os gradientes.
        # Todos os parâmetros do modelo deve ter sido setado para param.requires_grad = False
        loss.backward()            

        # Corte a norma dos gradientes para 1.0.
        # Isso ajuda a evitar o problema de "gradientes explosivos".
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
       
        # Atualize os parâmetros e dê um passo usando o gradiente calculado.
        # O otimizador dita a "regra de atualização" - como os parâmetros são
        # modificados com base em seus gradientes, taxa de aprendizagem, etc.
        optimizer.step()
                           
        # Atualize a taxa de aprendizagem.
        scheduler.step()

        del outputs
    
    # Média da perda do treinamento de todos os lotes da época.
    media_train_epoca_loss = np.mean(train_epoca_losses)

    # Acumule a perda de treinamento de todas as épocas para calcular a perda média do treinamento.    
    train_losses.append(media_train_epoca_loss)

    if model_args.use_wandb:
      wandb.log({"media_train_epoca_loss": media_train_epoca_loss})           
        
    # Medida de quanto tempo levou essa época.
    treinamento_epoca_total = formataTempo(time.time() - treinamento_epoca_t0)

    print("  Média perda(loss) do treinamento da época : {0:.8f}".format(media_train_epoca_loss))
    print("  Tempo de treinamento da época             : {:}".format(treinamento_epoca_total))    
    print("  Tempo parcial do treinamento              : {:} (h:mm:ss)".format(formataTempo(time.time()-treinamento_t0)))

    del py_input_ids
    del py_attention_masks
    del py_labels
    del train_epoca_losses
    del lote_treino_bar
  
  # Média da perda do treinamento de todas as épocas.
  media_train_loss = np.mean(train_losses)

  if model_args.use_wandb:
    wandb.log({"media_train_loss": media_train_loss})   

  print("  Média perda(loss) treinamento : {0:.8f}".format(media_train_loss))

  del train_losses
  del epoca_bar

  print("Treinamento completo!")

### Execução do Treinamento

Estamos prontos para iniciar o treinamento!

In [ ]:
# Registra o tempo inicial.
treinamento_t0 = time.time()

# Realiza o treinamento.
realizaTreinamento(documentos_treino, classes_treino, documentoids_treino, training_args.num_train_epochs)
  
# Medida de quanto tempo levou a execução do treinamento.
treinamento_total = formataTempo(time.time() - treinamento_t0)

print("  Tempo total treinamento       : {:}".format(treinamento_total))


Realizando Treinamento 


Criando Lotes Inteligentes de 9,960 amostras com tamanho de lote 4...

Tokenizando 9,960 amostra...
  Tokenizado 0 amostras.
  Tokenizado 1,000 amostras.
  Tokenizado 2,000 amostras.
  Tokenizado 3,000 amostras.
  Tokenizado 4,000 amostras.
  Tokenizado 5,000 amostras.
  Tokenizado 6,000 amostras.
  Tokenizado 7,000 amostras.
  Tokenizado 8,000 amostras.
  Tokenizado 9,000 amostras.
FEITO.
     9,960 amostras

     9,960 amostras após classificação

Criando lotes de tamanho 4...
  Selecionado 1,000 lotes.
  Selecionado 2,000 lotes.

  FEITO - Selecionado 2,490 lotes.

Preenchendo sequências dentro de cada lote...


  Lote   1,000  de    2,490.    Gasto: 0:02:14.  Restante: 0:03:19
  Lote   2,000  de    2,490.    Gasto: 0:04:27.  Restante: 0:01:05

  Média perda(loss) do treinamento da época : 0.40507782
  Tempo de treinamento da época             : 0:05:31
  Tempo parcial do treinamento              : 0:06:05 (h:mm:ss)
Criando Lotes Inteligentes de 9,960 amostras com tamanho de lote 4...

Tokenizando 9,960 amostra...
  Tokenizado 0 amostras.
  Tokenizado 1,000 amostras.
  Tokenizado 2,000 amostras.
  Tokenizado 3,000 amostras.
  Tokenizado 4,000 amostras.
  Tokenizado 5,000 amostras.
  Tokenizado 6,000 amostras.
  Tokenizado 7,000 amostras.
  Tokenizado 8,000 amostras.
  Tokenizado 9,000 amostras.
FEITO.
     9,960 amostras

     9,960 amostras após classificação

Criando lotes de tamanho 4...
  Selecionado 1,000 lotes.
  Selecionado 2,000 lotes.

  FEITO - Selecionado 2,490 lotes.

Preenchendo sequências dentro de cada lote...


  Lote   1,000  de    2,490.    Gasto: 0:02:13.  Restante: 0:03:18
  Lote   2,000  de    2,490.    Gasto: 0:04:23.  Restante: 0:01:04

  Média perda(loss) do treinamento da época : 0.25911422
  Tempo de treinamento da época             : 0:05:28
  Tempo parcial do treinamento              : 0:12:04 (h:mm:ss)
Criando Lotes Inteligentes de 9,960 amostras com tamanho de lote 4...

Tokenizando 9,960 amostra...
  Tokenizado 0 amostras.
  Tokenizado 1,000 amostras.
  Tokenizado 2,000 amostras.
  Tokenizado 3,000 amostras.
  Tokenizado 4,000 amostras.
  Tokenizado 5,000 amostras.
  Tokenizado 6,000 amostras.
  Tokenizado 7,000 amostras.
  Tokenizado 8,000 amostras.
  Tokenizado 9,000 amostras.
FEITO.
     9,960 amostras

     9,960 amostras após classificação

Criando lotes de tamanho 4...
  Selecionado 1,000 lotes.
  Selecionado 2,000 lotes.

  FEITO - Selecionado 2,490 lotes.

Preenchendo sequências dentro de cada lote...


  Lote   1,000  de    2,490.    Gasto: 0:02:12.  Restante: 0:03:16
  Lote   2,000  de    2,490.    Gasto: 0:04:23.  Restante: 0:01:04

  Média perda(loss) do treinamento da época : 0.14596666
  Tempo de treinamento da época             : 0:05:27
  Tempo parcial do treinamento              : 0:18:03 (h:mm:ss)
Criando Lotes Inteligentes de 9,960 amostras com tamanho de lote 4...

Tokenizando 9,960 amostra...
  Tokenizado 0 amostras.
  Tokenizado 1,000 amostras.
  Tokenizado 2,000 amostras.
  Tokenizado 3,000 amostras.
  Tokenizado 4,000 amostras.
  Tokenizado 5,000 amostras.
  Tokenizado 6,000 amostras.
  Tokenizado 7,000 amostras.
  Tokenizado 8,000 amostras.
  Tokenizado 9,000 amostras.
FEITO.
     9,960 amostras

     9,960 amostras após classificação

Criando lotes de tamanho 4...
  Selecionado 1,000 lotes.
  Selecionado 2,000 lotes.

  FEITO - Selecionado 2,490 lotes.

Preenchendo sequências dentro de cada lote...


  Lote   1,000  de    2,490.    Gasto: 0:02:11.  Restante: 0:03:15
  Lote   2,000  de    2,490.    Gasto: 0:04:22.  Restante: 0:01:04

  Média perda(loss) do treinamento da época : 0.10466218
  Tempo de treinamento da época             : 0:05:26
  Tempo parcial do treinamento              : 0:24:01 (h:mm:ss)

  Média perda(loss) treinamento : 0.22870522
Treinamento completo!
  Tempo total treinamento       : 0:24:01


# 5 Avaliação

Avaliando o modelo treinado no conjunto de dados de teste.

## 5.1 Função de Avaliação

In [ ]:
# Import das bibliotecas.
import torch
from tqdm.notebook import tqdm as tqdm_notebook

def realizaAvaliacao(documentos_teste, classes_teste, documentoids_teste):

  # Armazena o resultado da avaliação executada
  lista_resultado_avaliacao = []

  print("\nRealizando Avaliação: {}")

  # Predição no conjunto de teste no modelo.
  print('Predizendo rótulos para {:,} documentos de teste...'.format(len(documentos_teste)))

  # Use nossa nova função para preparar completamente nosso conjunto de dados.
  (py_input_ids, py_attention_masks, py_labels, documentosids) = cria_lotes_inteligentes(documentos_teste, classes_teste, documentoids_teste, training_args.per_device_eval_batch_size)

  # Escolha um intervalo para imprimir atualizações de progresso.
  intervalo_atualizacao = obter_intervalo_atualizacao(total_iteracoes=len(py_input_ids), numero_atualizacoes=10)

  # Coloque o modelo em modo de avaliação.
  model.eval()

  # Acumula as perdas da avaliação.
  test_losses = []

  # Acumula os resultados dos testes.
  vp = [] # Verdadeiro positivo
  vn = [] # Verdadeiro negativo
  fp = [] # Falso positivo
  fn = [] # Falso negativo

  # Barra de progresso dos lotes de teste.
  lote_teste_bar = tqdm_notebook(range(0, len(py_input_ids)), desc=f'Lotes ', unit=f'lotes', total=len(py_input_ids))

  # Para cada lote dos dados de avaliação(teste).
  for index in lote_teste_bar:

    # Progresso é atualizado a cada lotes, por exemplo, 100 lotes.
    if index % intervalo_atualizacao == 0 and not index == 0:        
        # Calcula o tempo gasto em minutos.
        tempoGasto = formataTempo(time.time() - avaliacao_t0)
        
        # Calculate the time tempoRestante based on our progress.
        passos_por_segundo = (time.time() - avaliacao_t0) / index
        segundos_restantes = passos_por_segundo * (len(py_input_ids) - index)
        tempoRestante = formataTempo(segundos_restantes)

        # Mostra o progresso.
        print('  Lote {:>7,}  de  {:>7,}.    Gasto: {:}.  Restando: {:}'.format(index, len(py_input_ids), tempoGasto, tempoRestante))
    
    # Copia o lote para a GPU.
    d_input_ids = py_input_ids[index].to(device)
    d_input_mask = py_attention_masks[index].to(device)
    d_labels = py_labels[index].to(device)
    d_documentoids = documentosids[index]

    # Diga a pytorch para não se preocupar em construir o gráfico de computação durante
    # o passe para frente, já que isso só é necessário para backprop (treinamento).
    with torch.no_grad():
        # Obtenha a saída de "logits" pelo modelo. Os "logits" são a saída
        # valores antes de aplicar uma função de ativação como o softmax.        
        # Retorno de model quando ´last_hidden_state=True´ é setado:    
        # last_hidden_state = outputs[0], pooler_output = outputs[1], hidden_states = outputs[2]
        outputs = model(d_input_ids,
                        token_type_ids=None, 
                        attention_mask=d_input_mask, 
                        labels=d_labels)
        
    # A perda(loss) é retornado em outputs[0] porque fornecemos rótulos(labels). 
    # É útil para comparar com a perda do treinamento, quando é realizado a avaliação entre as épocas de treinamento.
    loss = outputs[0]

    # E outputs[1] os "logits" - o modelo de saídas antes da ativação.
    # logits possui duas dimensões, a primeira do lote e a segunda do rótulo da predição
    logits = outputs[1]
        
    # Acumule a perda da avaliação em todos os lotes para que possamos
    # calcular a perda média no final. `loss` é um tensor contendo um único valor.
    # A função '.cpu()' move loss para a cpu.
    # A função `.item ()` retorna apenas o valor Python do tensor.
    test_losses.append(loss.cpu().item())

    # Recupera o indice do melhor resultado, maior valor dos tensores para coluna(1)
    _, classificacao = torch.max(logits, 1)

    # Verifica a classificação realizada e o rótulo previsto
    vp.append(((classificacao==1) & (d_labels==1)).sum().cpu().item())
    vn.append(((classificacao==0) & (d_labels==0)).sum().cpu().item())
    fp.append(((classificacao==1) & (d_labels==0)).sum().cpu().item())
    fn.append(((classificacao==0) & (d_labels==1)).sum().cpu().item())

    # Adiciona o documento de teste, o rótulo e a classificação realizada a lista de resultado
    for lote in range(len(d_labels)):
                
        lista_resultado_avaliacao.append([d_documentoids[lote],
                                d_labels[lote].cpu().item(), 
                                classificacao[lote].cpu().item()])
      
    del outputs

  # Soma as classificações realizadas
  vp_s, vn_s, fp_s, fn_s = sum(vp), sum(vn), sum(fp), sum(fn)

  # Acurácia indica uma performance geral do modelo. 
  # Dentre todas as classificações, quantas o modelo classificou corretamente(vp=1 e vn=0).
  acc = (vp_s+vn_s)/(vp_s+vn_s+fp_s+fn_s)

  # Recall(Revocação) avalia todas as situações da classe Positivo(vp=1) com o valor esperado e quantas estão corretas;
  if (vp_s+fn_s) != 0:
      rec = (vp_s)/(vp_s+fn_s)
  else:
      rec = 0

  # Precisão avalia as classificações da classe positivo(vp=1 e fp=0) que o modelo fez e quantas estão corretas.
  if (vp_s+fp_s) != 0:
      pre = (vp_s)/(vp_s+fp_s)
  else:
      pre = 0  

  # F1 é a média harmônica entre precisão e recall.
  if (pre + rec) != 0:  
    f1 = 2 * ((pre * rec)/(pre + rec))
  else:
    f1 = 0
  
  # Média da perda da avaliação
  media_test_loss = np.mean(test_losses)

  if model_args.use_wandb:
    # Log do wandb
    wandb.log({"acuracia": acc})
    wandb.log({"vp": vp_s})
    wandb.log({"vn": vn_s})
    wandb.log({"fp": fp_s})
    wandb.log({"fn": fn_s})
    wandb.log({"media_test_loss": media_test_loss})



  del py_input_ids
  del py_attention_masks
  del py_labels
  del test_losses
  del lote_teste_bar

  return media_test_loss, acc, rec, pre, f1, vp_s, vn_s, fp_s, fn_s, lista_resultado_avaliacao

## 5.2 Execução da Avaliação

In [ ]:
# Registra o tempo inicial.
avaliacao_t0 = time.time()

# Realiza a avaliação do modelo.
media_test_loss, acc, rec, pre, f1, vp_s, vn_s, fp_s, fn_s, lista_resultado_avaliacao = realizaAvaliacao(documentos_teste, classes_teste, documentoids_teste)

print('Avaliação loss           : {:.8f}; Acc: {:.8f}; Rec: {:.8f}; Pre: {:.8f}, F1:{:.8f}, vp: {:3d}; vn: {:3d}; fp: {:3d}; fn: {:3d}'.format( 
        media_test_loss, acc, rec, pre, f1, vp_s, vn_s, fp_s, fn_s))      

print("Acurácia                 : {:.8f}".format(acc))  

# Medida de quanto tempo levou a execução do treinamento e avaliação
avaliacao_total = formataTempo(time.time() - avaliacao_t0)

print("Tempo gasto na avaliação : {:}".format(avaliacao_total))


Realizando Avaliação: {}
Predizendo rótulos para 22,440 documentos de teste...
Criando Lotes Inteligentes de 22,440 amostras com tamanho de lote 8...

Tokenizando 22,440 amostra...
  Tokenizado 0 amostras.
  Tokenizado 2,000 amostras.
  Tokenizado 4,000 amostras.
  Tokenizado 6,000 amostras.
  Tokenizado 8,000 amostras.
  Tokenizado 10,000 amostras.
  Tokenizado 12,000 amostras.
  Tokenizado 14,000 amostras.
  Tokenizado 16,000 amostras.
  Tokenizado 18,000 amostras.
  Tokenizado 20,000 amostras.
  Tokenizado 22,000 amostras.
FEITO.
    22,440 amostras

    22,440 amostras após classificação

Criando lotes de tamanho 8...
  Selecionado 2,000 lotes.

  FEITO - Selecionado 2,805 lotes.

Preenchendo sequências dentro de cada lote...


  Lote     300  de    2,805.    Gasto: 0:00:47.  Restando: 0:06:30
  Lote     600  de    2,805.    Gasto: 0:00:56.  Restando: 0:03:27
  Lote     900  de    2,805.    Gasto: 0:01:07.  Restando: 0:02:21
  Lote   1,200  de    2,805.    Gasto: 0:01:17.  Restando: 0:01:43
  Lote   1,500  de    2,805.    Gasto: 0:01:27.  Restando: 0:01:16
  Lote   1,800  de    2,805.    Gasto: 0:01:37.  Restando: 0:00:54
  Lote   2,100  de    2,805.    Gasto: 0:01:46.  Restando: 0:00:36
  Lote   2,400  de    2,805.    Gasto: 0:01:56.  Restando: 0:00:20
  Lote   2,700  de    2,805.    Gasto: 0:02:06.  Restando: 0:00:05

Avaliação loss           : 3.07150027; Acc: 0.52508913; Rec: 0.07130125; Pre: 0.77145612, F1:0.13053765, vp: 800; vn: 10983; fp: 237; fn: 10420
Acurácia                 : 0.52508913
Tempo gasto na avaliação : 0:02:09


## 5.3 Salvando o resultado da classificação

In [ ]:
def salvaResultadoClassificacao(lista_resultado_avaliacao):

  if model_args.salvar_classificacao:

    # Import das bibliotecas.
    import os
    import datetime

    # Recupera a hora do sistema.
    data_e_hora = datetime.datetime.now()

    # Nome arquivo resultado
    NOME_ARQUIVO_CLASSIFICACAO = training_args.output_dir + MODELO_BERT + TAMANHO_BERT

    # Diretório para salvar o arquivo.
    DIRETORIO_CLASSIFICACAO = "/content/drive/MyDrive/Colab Notebooks/Data/CSTNEWS/validacao_classificacao/holdout/Classificacao/"

    # Verifica se o diretório existe
    if not os.path.exists(DIRETORIO_CLASSIFICACAO):  
      # Cria o diretório
      os.makedirs(DIRETORIO_CLASSIFICACAO)
      print('Diretório criado: {}'.format(DIRETORIO_CLASSIFICACAO))
    else:
      print('Diretório já existe: {}'.format(DIRETORIO_CLASSIFICACAO))

    # Nome do arquivo a ser aberto.
    NOME_ARQUIVO_CLASSIFICACAO_COMPLETO = DIRETORIO_CLASSIFICACAO + NOME_ARQUIVO_CLASSIFICACAO + ".csv"

    # Gera todo o conteúdo a ser salvo no arquivo
    novoConteudo = ""        
    for resultado in lista_resultado_avaliacao:      
      novoConteudo = novoConteudo + data_e_hora.strftime("%d/%m/%Y %H:%M") + ";" + str(resultado[0]) + ";" + str(resultado[1]) + ";" + str(resultado[2]) + "\n"

    # Verifica se o arquivo existe.
    if os.path.isfile(NOME_ARQUIVO_CLASSIFICACAO_COMPLETO):
      print("Atualizando arquivo classificação: {}".format(NOME_ARQUIVO_CLASSIFICACAO_COMPLETO))
      # Abre o arquivo para leitura.
      arquivo = open(NOME_ARQUIVO_CLASSIFICACAO_COMPLETO,'r')
      # Leitura de todas as linhas do arquivo.
      conteudo = arquivo.readlines()
      # Conteúdo a ser adicionado.
      conteudo.append(novoConteudo)

      # Abre novamente o arquivo (escrita).
      arquivo = open(NOME_ARQUIVO_CLASSIFICACAO_COMPLETO,'w')
      # escreva o conteúdo criado anteriormente nele.
      arquivo.writelines(conteudo)  
      # Fecha o arquivo.
      arquivo.close()
    else:
      print("Criando arquivo classificação: {}".format(NOME_ARQUIVO_CLASSIFICACAO_COMPLETO))
      # Abre novamente o arquivo (escrita).
      arquivo = open(NOME_ARQUIVO_CLASSIFICACAO_COMPLETO,'w')
      arquivo.writelines('data;id;classe;predicao\n' + novoConteudo)  # escreva o conteúdo criado anteriormente nele.
      # Fecha o arquivo.
      arquivo.close()


In [ ]:
salvaResultadoClassificacao(lista_resultado_avaliacao)

## 5.4 Salvando o resultado da avaliação

### Salva o resultado da avaliação 

Salva o resultado da avaliação do conjunto de dados de teste.

In [ ]:
def salvaResultadoAvaliacao():

  if model_args.salvar_avaliacao:

    # Import das bibliotecas.
    import os
    import datetime

    # Recupera a hora do sistema.
    data_e_hora = datetime.datetime.now()

    # Nome arquivo resultado
    NOME_ARQUIVO_AVALIACAO = training_args.output_dir + MODELO_BERT + TAMANHO_BERT

    # Diretório para salvar o arquivo de resultado.
    DIRETORIO_AVALIACAO = "/content/drive/MyDrive/Colab Notebooks/Data/CSTNEWS/validacao_classificacao/holdout/Avaliacao/"
  
    # Verifica se o diretório existe
    if not os.path.exists(DIRETORIO_AVALIACAO):  
      # Cria o diretório
      os.makedirs(DIRETORIO_AVALIACAO)
      print('Diretório criado: {}'.format(DIRETORIO_AVALIACAO))
    else:
      print('Diretório já existe: {}'.format(DIRETORIO_AVALIACAO))

    # Nome do arquivo a ser aberto.
    NOME_ARQUIVO_AVALIACAO_COMPLETO = DIRETORIO_AVALIACAO + NOME_ARQUIVO_AVALIACAO + ".csv"

    # Conteúdo a ser adicionado.
    novoConteudo = NOME_ARQUIVO_AVALIACAO + ";" + data_e_hora.strftime("%d/%m/%Y %H:%M") + ";"  + treinamento_total + ";"  + str(acc) + ";"  +  str(vp_s) + ";"  +  str(vn_s) + ";" +  str(fp_s) + ";" +  str(fn_s) + "\n"

    # Verifica se o arquivo existe.
    if os.path.isfile(NOME_ARQUIVO_AVALIACAO_COMPLETO):
      print("Atualizando arquivo resultado avaliação: {}".format(NOME_ARQUIVO_AVALIACAO_COMPLETO))
      # Abre o arquivo para leitura.
      arquivo = open(NOME_ARQUIVO_AVALIACAO_COMPLETO,'r')
      # Leitura de todas as linhas do arquivo.
      conteudo = arquivo.readlines()
      # Conteúdo a ser adicionado.
      conteudo.append(novoConteudo)

      # Abre novamente o arquivo (escrita).
      arquivo = open(NOME_ARQUIVO_AVALIACAO_COMPLETO,'w')
      # escreva o conteúdo criado anteriormente nele.
      arquivo.writelines(conteudo)  
      # Fecha o arquivo.
      arquivo.close()
    else:
      print("Criando arquivo resultado avaliação: {}".format(NOME_ARQUIVO_AVALIACAO_COMPLETO))
      # Abre novamente o arquivo (escrita).
      arquivo = open(NOME_ARQUIVO_AVALIACAO_COMPLETO,'w')
      arquivo.writelines('arquivo;data;tempo;acuracia;vp;vn;fp;fn\n' + novoConteudo)  # escreva o conteúdo criado anteriormente nele.
      # Fecha o arquivo.
      arquivo.close()

In [ ]:
salvaResultadoAvaliacao()

### Carrega e calcula a média da acurácia das execuções


In [ ]:
def carregaResultadoAvaliacao():

  # Import das bibliotecas.
  import os
  import pandas as pd

  # Acumuladores.
  somaAcuracia = 0
  listaTempo = []
  contaExecucoes = 0

  # Nome arquivo resultado
  NOME_ARQUIVO_AVALIACAO = training_args.output_dir + MODELO_BERT + TAMANHO_BERT

  # Diretório para salvar o arquivo.
  DIRETORIO_AVALIACAO = "/content/drive/MyDrive/Colab Notebooks/Data/CSTNEWS/validacao_classificacao/holdout/Avaliacao/"

  # Verifica se o diretório dos resultados existem.
  if os.path.exists(DIRETORIO_AVALIACAO):
    # Nome do arquivo mais o caminho
    NOME_ARQUIVO_AVALIACAO_COMPLETO = DIRETORIO_AVALIACAO + NOME_ARQUIVO_AVALIACAO + ".csv"
    # Verifica se o arquivo existe.
    if os.path.isfile(NOME_ARQUIVO_AVALIACAO_COMPLETO):
      # Carrega os dados do arquivo  
      dados = pd.read_csv(NOME_ARQUIVO_AVALIACAO_COMPLETO, sep=';')

      # Mostra os dados do teste da execução.
      for index, linha in dados.iterrows():
        
          # Cálculo das estatísticas
          acc = (linha['vp']+linha['vn'])/(linha['vp']+linha['vn']+linha['fp']+linha['fn'])
          if (linha['vp']+linha['fn']) != 0:
              rec = (linha['vp'])/(linha['vp']+linha['fn'])
          else:
              rec = 0
          if (linha['vp']+linha['fp']) != 0:
              pre = (linha['vp'])/(linha['vp']+linha['fp'])
          else:  
              pre = 0
          if (pre + rec) != 0:  
              f1 = 2 * ((pre * rec)/(pre + rec))
          else:
              f1 = 0
          qtdeTestes = linha['vp']+linha['vn']+linha['fp']+linha['fn']
          print('Arquivo: {}, Data: {}, Tempo:{}, QtdeTeste: {:3d}, Acc: {:.8f}, Rec: {:.8f}, Pre: {:.8f}, F1:{:.8f}, vp: {:4d}; vn: {:4d}; fp: {:4d}; fn: {:4d}'.format(
              linha['arquivo'], linha['data'], linha['tempo'], qtdeTestes, acc, rec, pre, f1, linha['vp'], linha['vn'], linha['fp'], linha['fn']))  
           
          # Guarda o tempo.
          listaTempo.append(str(linha['tempo']))

          # Procura a maior acurácia.
          somaAcuracia = somaAcuracia + acc

          # Conta o número de execuções.
          contaExecucoes = contaExecucoes + 1

      # Mostra a soma da acurácia . 
      print('Total acurácia                                          : {:.8f}'.format(somaAcuracia))
      # Mostra a quantidade de exeucões.
      print('Quantidade de execuções                                 : {}'.format(contaExecucoes))  
      # Calcula a média.
      media = somaAcuracia/contaExecucoes
      print('A média da acurácia de {:2d} execuções é                   : {:.8f}'.format(contaExecucoes, media))
      print('O tempo gasto na execução do treinamento {:2d} execuções é : {}'.format(contaExecucoes, somaTempo(listaTempo)))
      print('A média de tempo de {:2d} execuções é                      : {}'.format(contaExecucoes, mediaTempo(listaTempo)))
    else:
      print('Arquivo com os resultados não encontrado')    
  else:
    print('Diretório com os resultados não encontrado')  

In [ ]:
carregaResultadoAvaliacao()

Arquivo com os resultados não encontrado


# 6 Finalização

## 6.1 Salvando o Modelo para o wandb

In [ ]:
def salvaModeloWandb():
  
  if model_args.use_wandb and model_args.salvar_modelo_wandb:
  
    # Salva o modelo para o wandb    
    torch.save(model.state_dict(), os.path.join(wandb.run.dir, 'model_dict.pt'))

In [ ]:
salvaModeloWandb()

## 6.2 Salvando o Modelo Ajustado

Esta primeira célula (obtida de `run_glue.py` [aqui](https://github.com/huggingface/transformers/blob/35ff345fc9df9e777b27903f11fa213e4052595b/examples/run_glue.py#L495)) grava o modelo e o tokenizador no disco.

In [ ]:
def salvaModelo():
  
  if model_args.salvar_modelo:
  
    # Import de bibliotecas.
    import os

    # Salvando as melhores práticas: se você usar nomes padrão para o modelo, você pode recarregá-lo usando from_pretrained ()

    # Diretório de salvamento do modelo.
    output_dir = '/content/model_save/'

    # Cria o diretório de saída se necessário.
    if not os.path.exists(output_dir):
      os.makedirs(output_dir)

    print('Salvando o modelo para {}'.format(output_dir))

    # Salve um modelo treinado, configuração e tokenizer usando `save_pretrained ()`.
    # Eles podem então ser recarregados usando `from_pretrained ()`.
    model_to_save = model.module if hasattr(model, 'module') else model  # Cuide do treinamento distribuído/paralelo
    model_to_save.save_pretrained(output_dir)
    tokenizer.save_pretrained(output_dir)

    # Boa prática: salve seus argumentos de treinamento junto com o modelo treinado.
    torch.save (mode_args, os.path.join (output_dir, 'mode_args.bin'))
    torch.save (training_args, os.path.join (output_dir, 'training_args.bin'))

In [ ]:
salvaModelo()

Vamos verificar os tamanhos dos arquivos, por curiosidade.

In [ ]:
if model_args.salvar_modelo:
  !ls -l --block-size=K /content/model_save/

O maior arquivo é o peso do modelo, em torno de 416MB o base e 1.25G o large.

In [ ]:
if model_args.salvar_modelo:
  !ls -l --block-size=M /content/model_save/pytorch_model.bin

Para salvar seu modelo nas sessões do Colab Notebook, faça o download no seu computador local ou, idealmente, copie-o no seu Google Drive.

In [ ]:
if model_args.salvar_modelo:

  # Importando as bibliotecas.
  import os
  
  # Diretório local de salvamento do modelo.
  DIRETORIO_LOCAL_MODELO_AJUSTADO = '/content/modelo_ajustado/'

  # Diretório remoto de salvamento do modelo.  
  DIRETORIO_REMOTO_MODELO_AJUSTADO = "/content/drive/MyDrive/Colab Notebooks/Data/CSTNEWS/validacao_classificacao/holdout/modelo/modelo" + MODELO_BERT + TAMANHO_BERT

  # Verifica se o diretório existe
  if not os.path.exists(DIRETORIO_REMOTO_MODELO_AJUSTADO):  
    # Cria o diretório
    os.makedirs(DIRETORIO_REMOTO_MODELO_AJUSTADO)
    print('Diretório criado: {}'.format(DIRETORIO_REMOTO_MODELO_AJUSTADO))
  else:
    print('Diretório já existe: {}'.format(DIRETORIO_REMOTO_MODELO_AJUSTADO))

  ## Copia o arquivo do modelo para o diretório no Google Drive.
  !cp -r '$DIRETORIO_LOCAL_MODELO_AJUSTADO'* '$DIRETORIO_REMOTO_MODELO_AJUSTADO'

  print("Modelo copiado!")

## 6.3 Tempo final de processamento

Tempo processamento:  1:34:52 (h:mm:ss)

In [ ]:
 # Pega o tempo atual menos o tempo do início do processamento.
finalProcessamento = time.time()
tempoTotalProcessamento = formataTempo(finalProcessamento - inicioProcessamento)

print("")
print("  Tempo processamento:  {:} (h:mm:ss)".format(tempoTotalProcessamento))


  Tempo processamento:  0:29:51 (h:mm:ss)


Executa o wandb para finalizar a execução anterior

In [ ]:
if model_args.use_wandb:
  
    # Importando a biblioteca
    import wandb

    # Inicializando o registro do experimento
    # Na execução só pode existir de um init  para que não gere dois registros no wandb.
    wandb.init(project="ajustefinocstnews_avaliacaomoodle_v1_c_sb", name=training_args.output_dir)